# Hello, `notebook-http` mode

**_A notebook serving itself and an API from [fly.io](https://fly.io) using [Jupyter Kernel Gateway `notebook-http` mode](https://jupyter-kernel-gateway.readthedocs.io/en/latest/http-mode.html)_**

[Max Bo](https://maxbo.me), published 25 September 2024

---

I was inspired by the [new HATEOS article](https://htmx.org/essays/hateoas/) by the [htmx](https://htmx.org/) folks, and wanted to see if I could create a self-documenting API.

I think this fits the bill.

[github.com/MaxwellBo/hello-notebook-http-mode](https://github.com/MaxwellBo/hello-notebook-http-mode).


## Reflection

How does the notebook render itself?

In [57]:
# GET /
import os
import subprocess

if not os.path.exists('hello.html'):
    subprocess.run(["jupyter", "nbconvert", "--to", "html", "hello-notebook-http-mode.ipynb"])

with open('hello.html', 'r') as file:
    print(file.read())

In [58]:
# ResponseInfo GET /
import json

print(json.dumps({
    "headers" : {
        "Content-Type" : "text/html"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "text/html"}, "status": 200}


## Endpoints

Visit [/time](/time) to see the API response.

In [59]:
# GET /time

from datetime import datetime

print(datetime.now())

2024-09-25 12:41:25.821303


Query parameters can be extracted from an injected `REQUEST` object that's not available at standard Jupyter notebook evaluation time.

[/increment/:1](/increment/1)

In [60]:
# GET /increment/:value
import json

req = json.loads(REQUEST)
value = int(req['path']['value'])
print(value + 1)


NameError: name 'REQUEST' is not defined

## Docker

In [6]:
# GET /Dockerfile

with open('Dockerfile', 'r') as file:
    print(file.read())

FROM jupyter/datascience-notebook

WORKDIR /app

ADD . /app

# install the kernel gateway
RUN pip install jupyter_kernel_gateway

# run kernel gateway on container start, not notebook server
EXPOSE 8888

CMD ["jupyter", "kernelgateway",  "--KernelGatewayApp.api='kernel_gateway.notebook_http'", "--KernelGatewayApp.ip=0.0.0.0", "--KernelGatewayApp.port=8888", "--KernelGatewayApp.seed_uri=hello.ipynb", "--KernelGatewayApp.allow_origin='*'"]



To run locally:

```sh
docker build -t my/kernel-gateway .
docker run -it --rm -p 8888:8888 my/kernel-gateway
```

Adapted from [Jupyter Kernel Gatyway documentation - Running using a Docker stacks image](https://jupyter-kernel-gateway.readthedocs.io/en/stable/http-mode.html).

## fly.io

To deploy to [fly.io](https://fly.io):

```
fly launch
```

which generates

In [2]:
# GET /fly.toml

with open('fly.toml', 'r') as file:
    print(file.read())

# fly.toml app configuration file generated for hello-notebook-http-mode on 2024-09-25T09:29:23+10:00
#
# See https://fly.io/docs/reference/configuration/ for information about how to use this file.
#

app = 'hello-notebook-http-mode'
primary_region = 'syd'

[build]

[http_service]
  internal_port = 8888
  force_https = true
  auto_stop_machines = 'stop'
  auto_start_machines = true
  min_machines_running = 0
  processes = ['app']

[[vm]]
  memory = '1gb'
  cpu_kind = 'shared'
  cpus = 1



## Swagger

The Swagger API docs are available at [/_api/spec/swagger.json](/_api/spec/swagger.json).

## License

In [1]:
# GET /LICENSE

with open('LICENSE', 'r') as file:
    print(file.read())

MIT License

Copyright (c) 2024 Max Bo

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION W